네이버 및 구글 크롤링 부분 추가해야 함

In [2]:
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import time
import requests
import urllib3
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
ssl._create_default_https_context = ssl._create_unverified_context
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
%store -r voter_default

In [5]:
voter = voter_default[100:200]
voter

,이름,소속,직위,임시직위,이메일,핸드폰
100,김상수,용인대학교,조교수,교수,sgssk@naver.com,010-3234-5650
101,김상욱,충북대학교,교수,교수,sierra@chungbuk.ac.kr,010-5465-4432
102,김상원,국민대학교,학생,학생,kimsangwon@kookmin.ac.kr,010-8914-9620
103,김상진,동아대학교,교수,교수,skim10@dau.ac.kr,010-3034-0129
104,김상현,경북대학교,교수,교수,ksh@knu.ac.kr,010-6716-1836
...,...,...,...,...,...,...
195,김치헌,신한카드,부부장,부부장,chehukim@naver.com,010-4198-3250
196,김태경,경희대학교,부교수,교수,kimtk@kw.ac.kr,010-7279-1994
197,김태성,충북대학교,정교수,교수,kimts@cbnu.ac.kr,010-5391-2468
198,김태웅,성균관대학교,교수(은퇴),교수,tukim@skku.edu,010-9025-0452


---

In [ ]:
#전체 교수 정보 setting
prof_total = []
for i in range(len(voter)):
    prof_total.append([voter.iloc[i, 0], voter.iloc[i, 1]])
prof_total

[['박종화', '공주대학교'], ['박주석', '경희대학교 '], ['박준철', '강릉원주대학교 ']]

네이버 프로필 크롤링

In [ ]:
later_naver = [] #프로필이 없거나 오류가 난 행들 저장
#네이버 프로필 크롤링 위한 함수
def naver_crawl():
    try:
        url_naver = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={voter.iloc[i, 1]}+{voter.iloc[i, 0]}'
        res = requests.get(url_naver, headers=headers, verify=False).content
        prof_soup = BeautifulSoup(res, "html.parser")
    except:
        time.sleep(3)
        url_naver = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={voter.iloc[i, 1]}+{voter.iloc[i, 0]}'
        res = requests.get(url_naver, headers=headers, verify=False).content
        prof_soup = BeautifulSoup(res, "html.parser")    

    n = 0
    prof_naver = []
    while len(prof_soup.find_all('div', 'info_group')[n].find_all('dt')) > 0:
        n = n+1
    for j in range(n):
        prof_naver.append(prof_soup.find_all('div', 'info_group')[j].text)
    prof_total[i].append(prof_naver)
    if n == 0:
        later_naver.append(voter.iloc[i, 1]+" "+voter.iloc[i, 0])
        prof_total[i].remove([])

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#네이버 크롤링 실행
for i in range(len(voter)):
    try:
        naver_crawl()
    except:
        later_naver.append(voter.iloc[i, 1]+" "+voter.iloc[i, 0])
        continue
prof_total

In [ ]:
%store prof_total

Stored 'prof_total' (list)


구글 크롤링

검색 후 맨 위에 뜨는 페이지를 기준으로

In [ ]:
#selenium 위한 브라우저 열기
driver = webdriver.Chrome()
driver.implicitly_wait(2)
google_url = 'https://www.google.com/search?q=%27&sca_esv=564954462&hl=ko&source=hp&ei=c20BZf30OsCq0PEPjc-F0A0&iflsig=AO6bgOgAAAAAZQF7hIoHDk3NPXr59PTPTnC8ZegeGBsG&ved=0ahUKEwi9-vrCkKeBAxVAFTQIHY1nAdoQ4dUDCAs&uact=5&oq=%27&gs_lp=Egdnd3Mtd2l6IgEnSIErUMYbWLAocAR4AJABAJgBhgGgAYYBqgEDMC4xuAEDyAEA-AEBqAIA&sclient=gws-wiz&bshm=rimc/1'

In [ ]:
#search를 위한 함수
def google_search():
    driver.get(google_url)
    driver.find_element(By.ID, 'APjFqb').clear()

In [ ]:
#교수명으로부터 앞뒤로 텍스트 가져오되, 밑으로 행이 없을 경우 더 적은 행 가져오도록
def finder(x):
    #x에 들어가는 값: 해당 Html에서 교수명이 있는 index
    n = 0
    k = 0
    global list_
    list_ = []
    try:
        while n < 23:
            k += 1
            if len(page_text[x+k-1]) > 0:
                list_.append(page_text[x+k-2])
                n += 1
    except IndexError:
        while n < 4:
            k += 1
            if len(page_text[x+k-1]) > 0:
                list_.append(page_text[x+k-2])
                n += 1

In [ ]:
prof_google = [] #구글 크롤링 완료된 목록
later_google = [] #구글 크롤링 안 되거나 오류난 목록

for loop in range(len(voter)):
    google_search()
    #검색 후 맨 위 검색결과 클릭
    if '교수' in [voter.iloc[loop, 3]]:
        driver.find_element(By.ID, 'APjFqb').send_keys(voter.iloc[loop, 1] + " " + voter.iloc[loop, 0]+" "+'교수')
    else:
        driver.find_element(By.ID, 'APjFqb').send_keys(voter.iloc[loop, 1] + " " + voter.iloc[loop, 0])
    driver.find_element(By.CLASS_NAME, 'Tg7LZd').click()
    driver.find_element(By.CLASS_NAME, 'MjjYud').find_element(By.TAG_NAME, 'span').click()

    try:
        search_url = driver.current_url

        #들어간 사이트 크롤링 시도
        prof_soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_text = re.split(r'\n|\t|\r', prof_soup.text.replace(u'\xa0', u''))
        for i in range(page_text.count('')):
            page_text.remove('')

        #사이트명 추가
        title = prof_soup.find('title').text
        prof_total[loop].insert(2, title)

        #교수진 이름으로부터 아래 행들 크롤링
        name_dup = []
        for j in range(len(page_text)):
            if voter.iloc[loop, 0] in page_text[j]:
                name_dup.append(j)
                index1 = name_dup[0]
        if len(name_dup) > 1:
            index2 = name_dup[1]

        #홈페이지에 교수명 없을 경우
        if len(name_dup) == 0:
            prof_google.append([page_text])

        #교수명 1번 나올 경우    
        elif len(name_dup) == 1:
            finder(index1)
            prof_google.append(list_)

        #교수명 2번 나올 경우
        else:
            list__ = []
            finder(index1)
            list__ = [x for x in list_]
            finder(index2)
        #list__에 추가된 것이 교수 정보가 아닐 경우(교수명이 홈페이지 제목에 있을 경우 등)
            if '학사일정' in list_ or '시설 안내' in list_ or '인사말' in list_ or '행정실' in list_ or '공지사항' in list_ or 'Menu' in list_:
                list_ = []
            if '학사일정' in list__ or '시설 안내' in list__ or '인사말' in list__ or '행정실' in list__ or '공지사항' in list__ or 'Menu' in list_:
                list__ = []
                list__ = [x for x in list_]
            else:
                for n in range(len(list_)):
                    list__.append(list_[n])
            prof_google.append(list__)
    
    except IndexError:
        prof_google.append([page_text])
            
    except Exception as ex:
        prof_google.append('X')
        later_google.append([voter.iloc[loop, 1]+voter.iloc[loop, 0], f'{ex}'])
        print(f'{voter.iloc[loop, 1]+" "+voter.iloc[loop, 0]}', ex)


In [ ]:
%store prof_google
driver.quit()

Stored 'prof_google' (list)


In [ ]:
#구글 크롤링 결과 추가
for i in range(len(prof_total)):
    prof_total[i].insert(len(prof_total[i]), prof_google[i])
prof_total
#위 리스트 형식: [[이름, 소속, 사이트명(세부소속), [(있을 경우) 네이버 크롤링 결과], [구글 크롤링 결과]]]
#크롤링 결과 없을 경우 X 추가

크롤링 결과 글자 수 줄이기

In [ ]:
#구글 크롤링 결과 글자수 줄이기(글자수 초과되는 건들 한정)
edit1 = [] #글자수 수정된 목록
def cut_text_1():
    cut_num = len(prof_total[j][len(prof_total[j])-1])
    text_num = 0
    for i in range(cut_num):
        text_num += len(prof_total[j][len(prof_total[j])-1][i])
    if text_num > 9000:
        while text_num > 9000:
            del prof_total[j][len(prof_total[j])-1][-1]
            cut_num = len(prof_total[j][len(prof_total[j])-1])
            text_num = 0
            for i in range(cut_num):
                text_num += len(prof_total[j][len(prof_total[j])-1][i])
        print(text_num)
        edit1.append([j, text_num])
    else:
        print(text_num)

In [ ]:
#구글 크롤링 결과 글자수 줄이기(글자수 초과되는 건들 한정)
edit2 = [] #글자수 수정된 목록
def cut_text_2():
    cut_num = len(prof_total[j][len(prof_total[j])-1][0])
    text_num = 0
    for i in range(cut_num):
        text_num += len(prof_total[j][len(prof_total[j])-1][0][i])
    if text_num > 9000:
        while text_num > 9000:
            del prof_total[j][len(prof_total[j])-1][0][-1]
            cut_num = len(prof_total[j][len(prof_total[j])-1][0])
            text_num = 0
            for i in range(cut_num):
                text_num += len(prof_total[j][len(prof_total[j])-1][0][i])
        print(text_num)
        edit2.append([j, text_num])
    else:
        print(text_num)

In [ ]:
for j in range(len(prof_total)-1):
    cut_text_1()

In [ ]:
for j in range(len(prof_total)-1):
    try:
        cut_text_2()
    except IndexError:
        continue

---

뉴스기사 크롤링

관련도순 기사2 + 최근 기사

In [6]:
news_list = []
for i in range(len(voter)):
    news_list.append([])

In [7]:
#url 가져오는 함수
def get_url(x):
    global url_news
    if '학교' in voter.iloc[n, 1] and voter.iloc[n, 3] == '교수':
        url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1].replace("학교", "")}+"{voter.iloc[n, 0]}+교수"{x}'
    elif voter.iloc[n, 2] == '교수':
        url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1]}+"{voter.iloc[n, 0]}+교수"'
    else:
        url_news = f'https://search.naver.com/search.naver?where=news&query="{voter.iloc[n, 1]}+{voter.iloc[n, 0]}"'
    res = requests.get(url_news, headers=headers, verify=False).content
    global news_soup
    news_soup = BeautifulSoup(res, "html.parser")

In [19]:
#뉴스기사 크롤링
for n in range(1):
    %store news_list
    get_url('')
    news_num = len(news_soup.select('#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li', class_="bx"))
    if news_num > 3:
        try:
            nd = 1 # 뉴스 순서 (sp_nws)에 들어갈 숫자
            nt = 0 # nt가 2가 되면(기사 2개 가져오면) stop
            while nt < 2:
                try:
                    news_date = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_info > div.info_group > span').text
                    news_title = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a.news_tit').text
                    if '부고' in news_title:
                        nd += 1
                        continue
                    news1 = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a').attrs['href']
                    res = requests.get(news1, headers=headers, verify=False).content
                    news1_soup = BeautifulSoup(res, "html.parser")
                    news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                    news_list[n].append(news_title)
                    news_list[n].append(news_date)
                    news_list[n].append(news1)
                    news_list[n].append(news1_txt)
                    nt += 1
                    nd += 1
                except:
                    nd += 1

            get_url('&sort=1')

            nd = 1 # 뉴스 순서 (sp_nws)에 들어갈 숫자
            nt = 0 # nt가 2가 되면(기사 2개 가져오면) stop
            while nt < 2:
                if nd > 15:
                    break
                try:
                    news_date = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_info > div.info_group > span').text
                    news_title = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a.news_tit').text
                    if news_title in news_list[n] or news_title in news_list[n]:
                        nd += 1
                        continue
                    if '부고' in news_title:
                        nd += 1
                        continue
                    news1 = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a').attrs['href']
                    if news1 in news_list[n] or news1 in news_list[n]:
                        nd += 1
                        continue
                    res = requests.get(news1, headers=headers, verify=False).content
                    news1_soup = BeautifulSoup(res, "html.parser")
                    news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                    news_list[n].append(news_title)
                    news_list[n].append(news_date)
                    news_list[n].append(news1)
                    news_list[n].append(news1_txt)
                    nt += 1
                    nd += 1
                except:
                    nd += 1
        except Exception as ex:
            print(f'{voter.iloc[n, 1]+" "+voter.iloc[n, 0]}', ex)
    else:
        try:
            for num in range(news_num):
                news_date = news_soup.select_one(f'#sp_nws{num+1} > div > div > div.news_info > div.info_group > span').text
                news_title = news_soup.select_one(f'#sp_nws{num+1} > div > div > div.news_contents > a.news_tit').text
                news1 = news_soup.select_one(f'#sp_nws{num+1} > div > div > div.news_contents > a').attrs['href']
                res = requests.get(news1, headers=headers, verify=False).content
                news1_soup = BeautifulSoup(res, "html.parser")
                news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                news_list[n].append(news_title)
                news_list[n].append(news_date)
                news_list[n].append(news1)
                news_list[n].append(news1_txt)
        except Exception as ex:
            print(f'{voter.iloc[n, 1]+" "+voter.iloc[n, 0]}', ex)
    print(url_news)
    print(news_num)

Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=용인대 +"김상수+교수"
1


뉴스기사 글자 수 줄이가(10000자 이내로)

In [9]:
for i in range(len(news_list)):
    if len(news_list[i]) > 0:
        for j in range(1, int(len(news_list[i])/4+1), 1):
            if news_list[i][j*4-1].find('ⓒ') != -1:
                ni = news_list[i][j*4-1].find('ⓒ')
                news_list[i][j*4-1] = news_list[i][j*4-1][:ni]
            elif news_list[i][j*4-1].find('ⓒ') == -1 and news_list[i][j*4-1].find('right') != -1:
                ni = news_list[i][j*4-1].find('right')
                news_list[i][j*4-1] = news_list[i][j*4-1][:ni]
            else:
                news_list[i][j*4-1] = news_list[i][j*4-1]
            if len(news_list[i][j*4-1]) > 10000:
                news_list[i][j*4-1] = news_list[i][j*4-1][:10000]
                print(i)

5
7
26
40
44
53
90


In [16]:
news_list

[['용인대, 한국서비스경영학회 추계학술대회 우수발표 논문상 수상',
  '2021.12.06.',
  'http://www.newsis.com/view/?id=NISX20211206_0001677467&cID=10803&pID=14000',
  ''],
 ["충북대, 'K-MOOC'로 개발완료 강좌 결과보고회",
  '2017.08.21.',
  'http://www.veritas-a.com/news/articleView.html?idxno=93511',
  '충북대, \'K-MOOC\'로 개발완료 강좌 결과보고회 < 대학뉴스 < 대학 < 기사본문 - 베리타스알파주요서비스 바로가기본문 바로가기매체정보 바로가기로그인 바로가기기사검색 바로가기전체서비스 바로가기전체메뉴 버튼검색기사검색검색정기구독 신청하기구독체험뉴스스탠드로그인회원가입모바일웹로그인blogfacebooktwitterbandstorykakao GUESTSUBSCRIBER 구독자대입수시정시대학고입고교교육오피니언도서취업문화포토주요대학알림게시판소개UPDATE.2023-10-20 21:02 (금) 교육전문신문  베리타스알파02-564-6566구독신청고객센터구독 이벤트정기구독 FAQ자료실 발행일정지난신문보기  이전 기사보기다음 기사보기충북대, \'K-MOOC\'로 개발완료 강좌 결과보고회SNS 기사보내기이 기사를 공유합니다페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오스토리(으)로 기사보내기URL복사(으)로 기사보내기닫기스크롤 이동 상태바\xa0홈대학대학뉴스충북대, \'K-MOOC\'로 개발완료 강좌 결과보고회기자명 나동욱 기자 입력 2017.08.21 17:01 댓글 0SNS 기사보내기이 기사를 공유합니다페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오스토리(으)로 기사보내기URL복사(으)로 기사보내기닫기[베리타스알파=나동욱\xa0기자]\xa0충북대는\xa0교수학습지원센터가\xa024일\xa0교수학습지원센터\xa0스마트액티브러닝룸에서\xa02017년도\xa0K-MOOC\xa0강좌로\xa0선정된\xa

In [10]:
news_df = pd.DataFrame(news_list, columns = ['제목_1', '날짜_1', '링크_1', '내용_1', '제목_2', '날짜_2', '링크_2', '내용_2', '제목_3', '날짜_3', '링크_3', '내용_3', '제목_4', '날짜_4', '링크_4', '내용_4'])
news_df

,제목_1,날짜_1,링크_1,내용_1,제목_2,날짜_2,링크_2,내용_2,제목_3,날짜_3,링크_3,내용_3,제목_4,날짜_4,링크_4,내용_4
0,"용인대, 한국서비스경영학회 추계학술대회 우수발표 논문상 수상",2021.12.06.,http://www.newsis.com/view/?id=NISX20211206_00...,,None,None,None,None,None,None,None,None,None,None,None,None
1,"충북대, 'K-MOOC'로 개발완료 강좌 결과보고회",2017.08.21.,http://www.veritas-a.com/news/articleView.html...,"충북대, 'K-MOOC'로 개발완료 강좌 결과보고회 < 대학뉴스 < 대학 < 기사본...","충북대, 교직원 대상 '정부 3.0과 대학행정 활용' 교육",2016.08.31.,http://news.unn.net/news/articleView.html?idxn...,"충북대, 교직원 대상 ‘정부 3.0과 대학행정 활용’ 교육 < 대학뉴스 < 뉴스 <...","한국정책학회, 지역화폐 주제 토론회 개최",2021.12.02.,https://zdnet.co.kr/view/?no=20211202221333,"한국정책학회, 지역화폐 주제 토론회 개최 - ZDNet koreaZDNet Kore...",‘지역화폐 정책 이대로 좋은가’…정책학회 토론회 개최,2021.12.02.,http://www.dt.co.kr/contents.html?article_no=2...,‘지역화폐 정책 이대로 좋은가’…정책학회 토론회 개최 - 디지털타임스구독 ▶네이버 ...
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,동아대 '2022년 SW전문인재양성사업' 선정,2022.06.14.,http://www.veritas-a.com/news/articleView.html...,동아대 '2022년 SW전문인재양성사업' 선정 < 대학뉴스 < 대학 < 기사본문 -...,동아대 바이오헬스융합연구소 '의료/바이오/헬스 분야의 데이터사이언스 기술...,2020.11.24.,http://www.veritas-a.com/news/articleView.html...,동아대 바이오헬스융합연구소 '의료/바이오/헬스 분야의 데이터사이언스 기술 적용' 세...,부산 지산학협력 67호 브랜치 ㈜에스위너스에 개소,2023.09.13.,https://www.busaneconomy.com/news/articleView....,부산 지산학협력 67호 브랜치 ㈜에스위너스에 개소 < 경제 < 뉴스 < 기사본문 -...,"동아대, '2022년 SW전문인재양성사업' 선정",2022.06.16.,http://www.iusm.co.kr/news/articleView.html?id...,"동아대, '2022년 SW전문인재양성사업' 선정 < 부산 < 부산·경남 < 뉴스 <..."
4,"경북대 연구팀 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾아",2023.08.31.,http://www.veritas-a.com/news/articleView.html...,"경북대 연구팀 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾아 < 대학뉴...","경북대, 특허청 '지식재산 전문인력양성 중점대학' 사업 선정",2023.02.23.,http://www.kyongbuk.co.kr/news/articleView.htm...,"경북대, 특허청 '지식재산 전문인력양성 중점대학’ 사업 선정 - 경북일보 - 굿데이...",집먼지진드기로 인한 아토피피부염 치료법 국내 연구진 개발,2023.09.04.,https://kizmom.hankyung.com/news/view.html?aid...,집먼지진드기로 인한 아토피피부염 치료법 국내 연구진 개발 | Kizmom 뉴스 주...,"한국연구재단, 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾았다",2023.08.31.,https://www.jeonmae.co.kr/news/articleView.htm...,"한국연구재단, 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾았다 - 전국..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
96,"[대학가 투데이] 경희대, 건국대, 인하대, 광운대, 동아대",2022.05.10.,http://www.dhnews.co.kr/news/articleView.html?...,"[대학가 투데이] 경희대, 건국대, 인하대, 광운대, 동아대대학저널포항20.9℃통영...","[팩트체크] ""윤씨""...일본 언론 윤석열 대통령 하대했다?",2023.03.20.,https://www.newstof.com/news/articleView.html?...,"[팩트체크] ""윤씨""...일본 언론 윤석열 대통령 하대했다? < 팩트체크 < 정치 ...","위덕대, 문학 속의 원자력을 다룬 포럼 ‘원자력과 문학’ 개최",2022.11.30.,http://www.breaknews.com/936414,"위덕대, 문학 속의 원자력을 다룬 포럼 ‘원자력과 문학’ 개최시작페이지로 l 즐겨찾...","위덕대, 문학 속의 원자력 다룬 포럼 ‘원자력과 문학’ 개최",2022.11.30.,https://www.wikitree.co.kr/articles/811154,"위덕대, 문학 속의 원자력 다룬 포럼 ‘원자력과 문학’ 개최 | 위키트리위키트리 ..."
97,"충북대 보안컨설팅 융합전공 학생들, '보건의료 워킹그룹 모의 해킹메일 훈련...",2021.12.30.,http://www.veritas-a.com/news/articleView.html...,"충북대 보안컨설팅 융합전공 학생들, ‘보건의료 워킹그룹 모의 해킹메일 훈련’대회에서...","충북대 학생팀, 'HIRA 빅데이터 기반 산·학·관 연계 맞춤형 인재 양성 교육...",2020.09.04.,http://www.veritas-a.com/news/articleView.html...,"충북대 학생팀, 'HIRA 빅데이터 기반 산·학·관 연계 맞춤형 인재 양성 교육' ...","사이버위협 능동대응 기술 개발에 총 5년간 3,917억 원 투자 추진",2023.05.16.,https://www.dailysecu.com/news/articleView.htm...,"사이버위협 능동대응 기술 개발에 총 5년간 3,917억 원 투자 추진 - 데일리시큐...","제11회 CPS 보안워크숍, 9월 22일~23일 개최",2022.09.14.,https://www.dailysecu.com/news/articleView.htm...,"제11회 CPS 보안워크숍, 9월 22일~23일 개최 - 데일리시큐 ×전체기사산업이..."
98,서울대 교수들도 “집필 않겠다”…주요대학 모두 거부,A5면 TOP,http://www.hani.co.kr/arti/society/schooling/7...,,None,None,None,None,None,None,None,None,None,None,None,None


In [14]:
news_df_200 = news_df

In [37]:
%store news_list_200
%store voter
%store news_df_200

Stored 'news_list_200' (list)
Stored 'voter' (DataFrame)
Stored 'news_df_200' (DataFrame)
